# Show avg syllable duration across age

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from aging.plotting import (
    format_plots,
    save_factory,
    figure,
    PLOT_CONFIG,
    COLORMAPS,
    PlotConfig
)

In [2]:
format_plots()
c = PlotConfig()

In [3]:
from matplotlib.colors import LinearSegmentedColormap
# Define the color map
colors = ['#d94801','white','#008C8D']
custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
cmmf=custom_cmap

colors = ['#dadaeb','#6a51a3']
custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
cma=custom_cmap

colors = ['white','gray']
custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
cma=custom_cmap

colors = ['#ccece6','#99d8c9','#66c2a4','#41ae76','#238b45','#006d2c']
cml = sns.blend_palette(colors, n_colors=256, as_cmap=True)

# Define the color map
colors = ['#fee6ce','#d94801']
custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
cmf=custom_cmap

colors = ['#c7eae5','#008C8D']
custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
cmm=custom_cmap

## Females

In [4]:
df = pd.read_parquet('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/ontogeny_females_syllable_df_v00.parquet')
keep_syllables = np.loadtxt('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/to_keep_syllables_raw.txt', dtype=int)

In [5]:
df['idx'] = np.arange(len(df))
df.loc[df['onsets'], 'dur'] = df.query('onsets')['idx'].diff()
df['dur'] = df['dur'].ffill()
df = df.drop(columns=['idx'])

In [6]:
# for female:
uuids = ['f1d5dce5-f5cf-4b03-b07d-d9b8c1f034b0','6fa50ac7-6d49-4ef9-9059-3d90bdd8c5d4',
         'e9e05da8-fc59-40f3-a9c8-f57c492c6141','327e7fa1-2237-43d2-b25f-c1801912df33',
        '242bee8e-0ee7-45e7-8a13-678836a4cddb','552ec70f-0676-4471-8ab0-403e1fcf43e3','d6f254af-d55a-427a-96e0-c452a233cbe2']
age_mapping = {
    93: 94,
    95: 94,
    97: 98,
    98: 98,
    102: 98,
    103: 105,
    105: 105,
    107: 105
}

df['age'] = df['age'].map(age_mapping).fillna(df['age'])
df.loc[df['uuid'].isin(uuids), 'age'] = 52
df=df.query('age<127')

In [7]:
durs_all = df.groupby(['age', 'mouse','syllables'])['dur'].mean() / 30
durs_all = durs_all.unstack('syllables')
durs_all = durs_all[keep_syllables]

In [8]:
durs = pd.DataFrame(durs_all.mean(axis=1), columns=['dur'])

In [9]:
fig, ax = plt.subplots()
fig.set_size_inches(1.3,1.3) 
plt.scatter(data=durs.reset_index(),
            x='age', 
            y='dur', 
            s=10, 
            c='age', 
            edgecolor='none',
            linewidth=0.25,
            alpha=0.75,
            cmap=cmf)

ax = sns.regplot(
    data=durs.reset_index(),
    x="age",
    y="dur",
    #color=COLORMAPS.ont_male(255),
    color='black',
    #scatter_kws=dict(linewidths=0),
    scatter=False,
    logx=True,
)
ax.set(xlabel="Age (weeks)", ylabel="Syllable duration", ylim=[0.45,0.8])
sns.despine()

In [10]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'syll_duration_female_log.pdf')

In [18]:
fig, ax = plt.subplots()
fig.set_size_inches(1.3,1.3) 
plt.scatter(data=durs.reset_index(),
            x='age', 
            y='dur', 
            s=10, 
            c='age', 
            edgecolor='none',
            linewidth=0.25,
            alpha=0.75,
            cmap=cmf)

ax = sns.regplot(
    data=durs.reset_index(),
    x="age",
    y="dur",
    #color=COLORMAPS.ont_male(255),
    color='black',
    #scatter_kws=dict(linewidths=0),
    scatter=False,
    #logx=True,
)
ax.set(xlabel="Age (weeks)", ylabel="Syllable duration", ylim=[0.45,0.8], xticks=[20,40,60,80,100])
sns.despine()

In [19]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'syll_duration_female_linear.pdf')

In [13]:
dur_df = durs.reset_index()
dur_bins = pd.cut(dur_df['age'], 12, labels=False)
dur_df['bins'] = dur_bins
age_map = dur_df.groupby('bins')['age'].mean()
dur_df['avg_age'] = dur_df['bins'].map(age_map).round(0)

In [14]:
ax = sns.boxplot(
    data=dur_df.reset_index(),
    x="avg_age",
    y="dur",
    showcaps=False,
    showfliers=False,
    color=COLORMAPS.ont_female(255),
)
ax.set(xlabel="Binned age (weeks)", ylabel="Syllable duration (s)")
sns.despine()